In [216]:
!pip install --upgrade pythainlp
!pip install pyLDAvis
!pip install --upgrade pandas==1.2

In [217]:
import pandas as pd
import pythainlp
from pythainlp.corpus import thai_words
import gensim
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [218]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [219]:
df = pd.read_csv('/content/drive/MyDrive/crm/CustomerReviews.csv')
print(df.head(5))

   Review ID  ... Rating
0          1  ...    5.0
1          2  ...    5.0
2          3  ...    4.0
3          4  ...    4.0
4          5  ...    5.0

[5 rows x 7 columns]


In [220]:
def tokenize_with_space(sentence):
  merged = ''
  words = pythainlp.word_tokenize(str(sentence), engine='newmm')
  for word in words:
    merged = merged + ',' + word
  return merged[1:]

In [221]:
df['Review_tokenized'] = df['Review'].apply(lambda x: tokenize_with_space(x))
df.head(5)

,Review ID,Restaurant_ID,Restaurant,User,Headline,Review,Rating,Review_tokenized
0,1,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,7b16469831074f7abc7824745ee75212,ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักส...,ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักส...,5.0,"ที่,สำคัญ,ของ,ร้าน,นี้,คือ,บริการ,ดีมาก,พนักงา..."
1,2,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,pakkaramonpondej,รสชาติเหมือนทุกสาขา แต่สาขานี้ บริการดี ที่นั่งดี,นึกถึงชาบูญี่ปุ่นยังไงก็ต้อง คิดถึงโมโม่ พาราไ...,5.0,"นึกถึง,ชาบู,ญี่ปุ่น,ยังไง,ก็,ต้อง, ,คิดถึง,โม,..."
2,3,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,saanowy,ชาบูพรีเมี่ยมสไตล์ญี่ปุ่น เนื้อดีมากกก,มาทานช่วงนี้ สามารถนั่งโต๊ะเดียวกัน หม้อเดียวก...,4.0,"มา,ทาน,ช่วงนี้, ,สามารถ,นั่ง,โต๊ะ,เดียวกัน, ,ห..."
3,4,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ployynp,เนื้อดี ไอติมดี คุ้มค่าดี,ถ้านึกถึงชาบูที่มีเนื้อเน้นๆ ในราคาไม่โหดจนเกิ...,4.0,"ถ้า,นึกถึง,ชาบู,ที่,มี,เนื้อ,เน้น,ๆ, ,ใน,ราคา,..."
4,5,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,665a902b335b434ab489f00f2fbb477e,อาหารมีคุณภาพ บริการดีค่ะ,เดินมาหน้าร้านแล้วได้กลิ่นชาบูหอมมาก ๆ ประกอบ...,5.0,"เดิน,มา,หน้า,ร้าน,แล้ว,ได้กลิ่น,ชาบู,หอ,มมาก, ..."


In [222]:
rest_lst = list(df['Restaurant'].unique())
rest_lst

['Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ',
 'Shabushi (ชาบูชิ) เดอะมอลล์บางกะปิ ชั้น G',
 'ข้าน้อยขอชาบู']

In [223]:
df = df[df['Restaurant']=='Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ']

stop_words = list(pythainlp.corpus.thai_stopwords())
num = [str(i) for i in range(0,1000)]
documents = df['Review_tokenized'].to_list()
texts = [[text.strip().lower() for text in doc.split(',') if len(text) != 1 and text not in stop_words and text not in num] for doc in documents]
dictionary = gensim.corpora.Dictionary(texts)

print(dictionary.token2id.keys())

dict_keys(['กิน', 'ข้างนอก', 'ความต้องการ', 'คอ', 'คุณภาพ', 'คุ้ม', 'จำกัด', 'ซุป', 'ดำ', 'ดี', 'ดีมาก', 'ถาด', 'ถาม', 'น่ารัก', 'น้ำ', 'น้ำจิ้ม', 'บริการ', 'บาท', 'ประทับใจ', 'พนักงาน', 'พอกับ', 'ภาพ', 'รสชาติ', 'ราคา', 'ร้าน', 'สรุป', 'สะอาดสะอ้าน', 'สัน', 'หลากหลาย', 'อร่อย', 'อันนี้', 'อาหาร', 'เคลม', 'เซ็ท', 'เต้าเจี้ยว', 'เนื้อ', 'เป็ด', 'เผ็ด', 'แบบนี้', 'แปลก', 'แฟน', 'โดยรวม', 'โมจิ', 'ใส่ใจ', 'ไม่ต้อง', 'ไอติม', '', 'ctw', 'terminal', 'กา', 'การงด', 'คิดถึง', 'ชาบู', 'ญี่ปุ่น', 'ฐาน', 'ทะยอย', 'ทำงาน', 'ที่ทาง', 'นาที', 'นึกถึง', 'พนักงานบริการ', 'พระราม', 'พรีเมียม', 'พาราไดซ์', 'มาตรา', 'มีโอ', 'รายละเอียด', 'ลูกค้า', 'สมาทาน', 'สาขา', 'สู้', 'ออร์เดอร์', 'เครื่องดื่ม', 'เค้า', 'เจอ', 'เป็นกันเอง', 'เหวี่ยง', 'แจ้ง', 'โม', 'โม่', 'โอเค', '++', '::', 'กก', 'กรุบ', 'กั้น', 'ของหวาน', 'คนละ', 'คาว', 'คุ', 'คุ้มค่า', 'จาน', 'จิ้ม', 'จืด', 'ฉาก', 'ชอบ', 'ชาเขียว', 'ชิ้น', 'ซอย', 'ซอส', 'ตก', 'ตะ', 'ตัก', 'ตามน้ำ', 'ต่างกัน', 'ถั่ว', 'ถ่ายรูป', 'ทาน', 'นั่ง', 'นุ่ม', 'บด', 'บาร์'

In [224]:
gensim_corpus = [dictionary.doc2bow(text, allow_update=True) for text in texts]
print(gensim_corpus)

[[(0, 5), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 2), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 3), (17, 1), (18, 2), (19, 1), (20, 1), (21, 1), (22, 3), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 3), (30, 1), (31, 3), (32, 1), (33, 1), (34, 1), (35, 2), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1)], [(4, 1), (9, 2), (13, 1), (16, 2), (19, 1), (24, 4), (31, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 3), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1)], [(0, 1), (5, 1), (7, 5), (8, 3), (14, 8), (15, 2), (19, 1), (22, 3), (23, 1), (25, 1), (29, 4), (35, 3), (36, 2), (45, 2), (46, 6), (52, 3), (53, 1), (81, 1), (82, 2), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (8

In [225]:
word_frequencies = [[(dictionary[id], frequence) for id, frequence in couple] for couple in gensim_corpus]
print(word_frequencies)

[[('กิน', 5), ('ข้างนอก', 1), ('ความต้องการ', 1), ('คอ', 1), ('คุณภาพ', 1), ('คุ้ม', 1), ('จำกัด', 1), ('ซุป', 1), ('ดำ', 1), ('ดี', 2), ('ดีมาก', 1), ('ถาด', 1), ('ถาม', 1), ('น่ารัก', 1), ('น้ำ', 1), ('น้ำจิ้ม', 1), ('บริการ', 3), ('บาท', 1), ('ประทับใจ', 2), ('พนักงาน', 1), ('พอกับ', 1), ('ภาพ', 1), ('รสชาติ', 3), ('ราคา', 1), ('ร้าน', 1), ('สรุป', 1), ('สะอาดสะอ้าน', 1), ('สัน', 1), ('หลากหลาย', 1), ('อร่อย', 3), ('อันนี้', 1), ('อาหาร', 3), ('เคลม', 1), ('เซ็ท', 1), ('เต้าเจี้ยว', 1), ('เนื้อ', 2), ('เป็ด', 1), ('เผ็ด', 1), ('แบบนี้', 1), ('แปลก', 1), ('แฟน', 1), ('โดยรวม', 1), ('โมจิ', 1), ('ใส่ใจ', 1), ('ไม่ต้อง', 1), ('ไอติม', 1)], [('คุณภาพ', 1), ('ดี', 2), ('น่ารัก', 1), ('บริการ', 2), ('พนักงาน', 1), ('ร้าน', 4), ('อาหาร', 1), ('', 1), ('ctw', 1), ('terminal', 1), ('กา', 1), ('การงด', 1), ('คิดถึง', 1), ('ชาบู', 1), ('ญี่ปุ่น', 1), ('ฐาน', 1), ('ทะยอย', 1), ('ทำงาน', 1), ('ที่ทาง', 1), ('นาที', 1), ('นึกถึง', 1), ('พนักงานบริการ', 1), ('พระราม', 1), ('พรีเมียม', 1), ('พาราได

In [226]:
num_topics = 3
chunksize = 4000 # size of the doc looked at every pass
passes = 20 # number of passes through documents
iterations = 50
eval_every = 1  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = gensim.models.LdaModel(corpus=gensim_corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)

CPU times: user 140 ms, sys: 2.89 ms, total: 142 ms
Wall time: 145 ms


In [227]:
pyLDAvis.gensim_models.prepare(model, gensim_corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.026883 -0.006833       1        1  51.032168
0      0.026275 -0.007318       2        1  33.469527
2      0.000608  0.014151       3        1  15.498305, topic_info=       Term       Freq      Total Category  logprob  loglift
24     ร้าน  11.000000  11.000000  Default  30.0000  30.0000
35    เนื้อ  11.000000  11.000000  Default  29.0000  29.0000
46           12.000000  12.000000  Default  28.0000  28.0000
52     ชาบู  11.000000  11.000000  Default  27.0000  27.0000
0       กิน   7.000000   7.000000  Default  26.0000  26.0000
..      ...        ...        ...      ...      ...      ...
15  น้ำจิ้ม   0.744824   6.621961   Topic3  -4.6123  -0.3206
9        ดี   0.610849   5.209377   Topic3  -4.8106  -0.2789
8        ดำ   0.605721   5.581486   Topic3  -4.8190  -0.3564
31    อาหาร   0.541837   4.523729   Topic3  -4.9304  -0.2577
19  พนักงาน   0.495109   4.050426   Topic3  -5.0206  -0.2374

[214 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
46        1  0.558360         
46        2  0.239297         
46        3  0.159531         
201       1  0.874794      4.0
82        1  0.504934       ::
...     ...       ...      ...
157       1  0.503377      ไผ่
44        2  0.926843  ไม่ต้อง
158       1  0.506731      ไม้
45        1  0.497868    ไอติม
45        2  0.248934    ไอติม

[185 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 3])

In [228]:
model.show_topic(0)

[('ร้าน', 0.031236228),
 ('น้ำ', 0.026176317),
 ('เนื้อ', 0.022195807),
 ('กิน', 0.022095622),
 ('ชาบู', 0.021894583),
 ('รสชาติ', 0.020991592),
 ('อร่อย', 0.019517416),
 ('', 0.018295007),
 ('บริการ', 0.017127255),
 ('ดี', 0.017049478)]

In [229]:
model.show_topic(1)

[('น้ำ', 0.034599114),
 ('', 0.029673275),
 ('ชาบู', 0.023906127),
 ('เนื้อ', 0.02239853),
 ('เลือก', 0.020494068),
 ('อร่อย', 0.02011969),
 ('ร้าน', 0.01949885),
 ('ซุป', 0.01687516),
 ('รสชาติ', 0.016078204),
 ('น้ำจิ้ม', 0.015736103)]

In [232]:
model.show_topic(2)

[('', 0.02991381),
 ('เนื้อ', 0.029619077),
 ('ร้าน', 0.027986385),
 ('ชาบู', 0.02671567),
 ('น้ำ', 0.024342082),
 ('อร่อย', 0.02128779),
 ('เลือก', 0.016747663),
 ('ชอบ', 0.016557677),
 ('สาขา', 0.015346066),
 ('กิน', 0.014302666)]

In [230]:
df['topics'] = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][0])
df['score'] = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][1])

In [231]:
df[['Restaurant', 'Review', 'topics','score']]

,Restaurant,Review,topics,score
0,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักส...,0,0.999052
1,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,นึกถึงชาบูญี่ปุ่นยังไงก็ต้อง คิดถึงโมโม่ พาราไ...,0,0.998744
2,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,มาทานช่วงนี้ สามารถนั่งโต๊ะเดียวกัน หม้อเดียวก...,1,0.999619
3,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ถ้านึกถึงชาบูที่มีเนื้อเน้นๆ ในราคาไม่โหดจนเกิ...,2,0.997777
4,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,เดินมาหน้าร้านแล้วได้กลิ่นชาบูหอมมาก ๆ ประกอบ...,2,0.998332
5,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ร้านบุฟเฟ่ ชาบูแนวญี่ปุ่น สายเนื้อหมู เนื้อวัว...,1,0.998263
6,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,Number 20 : โมโม – พาราไดส์ (สาขาเดอะมอลบางกะป...,1,0.999010
7,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ร้านชาบูเฟรนไชส์รสชาติดีมากคุ้มค่าเหมาะสมกับรา...,0,0.998889
